***Problem 1-Carseats Linear Regression***

**1(a) Fit a multiple linear regression model to predict Sales using Price, Urban, and US**

In [593]:
#데이터 관리 및 조작
import pandas as pd
df=pd.read_csv('/Users/lavi/MLDL_25_2_ASSIGNMENT/ASSIGNMENT_1/Data/Carseats.csv')

#Check Files for ensure
print(df.head())


   Sales  CompPrice  Income  Advertising  Population  Price ShelveLoc  Age  \
0   9.50        138      73           11         276    120       Bad   42   
1  11.22        111      48           16         260     83      Good   65   
2  10.06        113      35           10         269     80    Medium   59   
3   7.40        117     100            4         466     97    Medium   55   
4   4.15        141      64            3         340    128       Bad   38   

   Education Urban   US  
0         17   Yes  Yes  
1         10   Yes  Yes  
2         12   Yes  Yes  
3         14   Yes  Yes  
4         13   Yes   No  


In [594]:
#변수 설정
Y=df['Sales']
X=df[['Price', 'Urban','US']]

#Check data for ensure
print(Y.head())
print(X.head())

0     9.50
1    11.22
2    10.06
3     7.40
4     4.15
Name: Sales, dtype: float64
   Price Urban   US
0    120   Yes  Yes
1     83   Yes  Yes
2     80   Yes  Yes
3     97   Yes  Yes
4    128   Yes   No


In [595]:
# Quali. var.(Urban, US) 인코딩
X_encoded = pd.get_dummies(X, drop_first=True)
print(X_encoded.head())

   Price  Urban_Yes  US_Yes
0    120       True    True
1     83       True    True
2     80       True    True
3     97       True    True
4    128       True   False


In [596]:
# 머신러닝 (Using Scikit-Learn)
#Scikit-learn Developers (2024). “Linear Regression.” https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html 인용

from sklearn.linear_model import LinearRegression

lin_model=LinearRegression(fit_intercept=True)
lin_model.fit(X_encoded,Y)

r2=lin_model.score(X_encoded,Y)

# Output Check
print("Intercept: ",lin_model.intercept_)
print("Parameters: ", lin_model.coef_)
print("R_square: ", r2)


Intercept:  13.043468936764892
Parameters:  [-0.05445885 -0.02191615  1.2005727 ]
R_square:  0.23927539218405547


*1(a),(b) Report by pdf file.*

*(a) Which contains the fitted coefficients (including the intercept) and the training R2 of the model.*

*(b) Write the fitted model in equation form, and Provide an interpretation of each coefficient in the model.*

**1(c) Closed-form OLS and comparison**

In [597]:
#강의자료 Lecture2, p.30 인용
import numpy as np
n=len(X_encoded)
X_closed = np.column_stack((np.ones(n), X_encoded)).astype(float)

Y_closed=Y.values.reshape(-1,1)

#check dimension X:n*(p+1), y:(n*1)
print(X_closed.shape, Y_closed.shape)

(400, 4) (400, 1)


In [598]:
beta_hat=np.linalg.inv(X_closed.T @ X_closed) @ X_closed.T @ Y_closed

print(beta_hat)

[[13.04346894]
 [-0.05445885]
 [-0.02191615]
 [ 1.2005727 ]]


In [599]:
coef_sklearn = np.concatenate(([lin_model.intercept_], lin_model.coef_))

comparison = pd.DataFrame({
    'OLS_closed_form': beta_hat.flatten(),
    'sklearn': coef_sklearn,
})
comparison['abs_diff'] = abs(comparison['OLS_closed_form'] - comparison['sklearn'])
print(comparison)

   OLS_closed_form    sklearn      abs_diff
0        13.043469  13.043469  3.907985e-14
1        -0.054459  -0.054459  1.457168e-16
2        -0.021916  -0.021916  2.942091e-15
3         1.200573   1.200573  1.998401e-15


**1(d) test H0 : βj = 0 versus H1 : βj != 0 at level α = 0.05**

In [600]:
#SciPy Documentation (2024). “scipy.stats.t” https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html
#및 강의자료 Lecture 3, p.8인용
from scipy import stats

Y_hat=X_closed @ beta_hat
#sci-kit의 LinRegression으로 도출된 coef_sklearn과 closed form의 해석적 해 모두 사용하여도 된다.
# 두 값이 거의 동일하므로... 다만 coef_sklearn은 (4,)의 차원이므로 (4,1)의 차원으로 reshape(-1,1)필요함.
epsilon=Y_closed-Y_hat

n,p_plus_1=X_closed.shape
df=n-p_plus_1

sigma_sq=(epsilon.T @ epsilon)/df
sigma=np.sqrt(sigma_sq)

var=sigma_sq*np.linalg.inv(X_closed.T @ X_closed)
se=np.sqrt(np.diag(var))

t_stat=beta_hat.flatten()/se
p_values = 2 * (1 - stats.t.cdf(np.abs(t_stat), df))


alpha = 0.05
significant = p_values < alpha

colnames = ['Intercept', 'Price', 'Urban_Yes', 'US_Yes']

t_table = pd.DataFrame({
    'Estimate': beta_hat.flatten(),
    'Std.Error': se,
    't value': t_stat,
    'Pr(>|t|)': p_values,
    'Significant (α=0.05)': significant
}, index=colnames)

print(t_table)

            Estimate  Std.Error    t value  Pr(>|t|)  Significant (α=0.05)
Intercept  13.043469   0.651012  20.035674  0.000000                  True
Price      -0.054459   0.005242 -10.389232  0.000000                  True
Urban_Yes  -0.021916   0.271650  -0.080678  0.935739                 False
US_Yes      1.200573   0.259042   4.634673  0.000005                  True


**1(e) Perform a validation set approach(7:3 random val & 5-fold cross val)**

In [601]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error

random_seed = 1

X_train, X_val, y_train, y_val = train_test_split(
    X_encoded, Y, test_size=0.3, random_state=random_seed
)

#Check 7:3 sizing
print(f"Train size: {len(X_train)}, Validation size: {len(X_val)}")

Train size: 280, Validation size: 120


*7:3 Validation*

In [602]:
#Scikit-learn Developers (2024). “train_test_split.” https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html 인용
lm = LinearRegression(fit_intercept=True)
lm.fit(X_train, y_train)

y_pred_val = lm.predict(X_val)

r2_val = r2_score(y_val, y_pred_val)
mse_val = mean_squared_error(y_val, y_pred_val)

print(f"Validation[randomseed={random_seed}] R²: {r2_val:.4f}")
print(f"Validation[randomseed={random_seed}] MSE: {mse_val:.4f}")


Validation[randomseed=1] R²: 0.2849
Validation[randomseed=1] MSE: 5.6138


*5-fold Cross Validation*

In [603]:
#Scikit-learn Developers (2024). “cross_val_score.” https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html 인용
lm_cv = LinearRegression(fit_intercept=True)

r2_cv_scores = cross_val_score(lm_cv, X_encoded, Y, cv=5, scoring='r2')
mse_cv_scores = -cross_val_score(lm_cv, X_encoded, Y, cv=5, scoring='neg_mean_squared_error')

r2_cv_mean = np.mean(r2_cv_scores)
mse_cv_mean = np.mean(mse_cv_scores)

print(f"Cross-validated R² (mean): {r2_cv_mean:.4f}")
print(f"Cross-validated MSE (mean): {mse_cv_mean:.4f}")

Cross-validated R² (mean): 0.2038
Cross-validated MSE (mean): 6.1903


***Problem 2(Logistic Regression)***

**2(a) LogisticRegression, fit a model to predict default from income and balance.**

In [604]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss

In [605]:
df = pd.read_csv("/Users/lavi/MLDL_25_2_ASSIGNMENT/ASSIGNMENT_1/Data/Default.csv")

print(df.head())

  default student      balance        income
0      No      No   729.526495  44361.625074
1      No     Yes   817.180407  12106.134700
2      No      No  1073.549164  31767.138947
3      No      No   529.250605  35704.493935
4      No      No   785.655883  38463.495879


In [606]:
df['default_bin'] = df['default'].map({'No': 0, 'Yes': 1})
X = df[['income', 'balance']]
y = df['default_bin']

std_balance = df['balance'].std()
mean_balance=df['balance'].mean()
std_income = df['income'].std()
mean_income=df['income'].mean()

print("Balance Mean:", mean_balance)
print("Income Mean:", mean_income)

print("Balance SE:", std_balance)
print("Income SE:", std_income)

Balance Mean: 835.3748856125571
Income Mean: 33516.981875960504
Balance SE: 483.7149852056177
Income SE: 13336.639562689912


In [607]:
RANDOM_SEED=1
np.random.seed(RANDOM_SEED)

In [608]:
#Scikit-learn Developers (2024). “Logistic Regression.” https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html 인용

#Scikit-learn Developers (2024). “Pipeline.” https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html 인용

pipe = Pipeline([
    ('scaler', StandardScaler()),  # 표준화 여부 O(StandardScaler)
    ('clf', LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000, random_state=RANDOM_SEED))
])

In [609]:
pipe.fit(X, y)
clf = pipe.named_steps['clf']
intercept = clf.intercept_[0]
coef = clf.coef_[0]


In [610]:
p_train = pipe.predict_proba(X)[:, 1]

In [611]:
#Scikit-learn Developers (2024). “log_loss.” https://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html 인용
n = len(y)
train_loglik = -log_loss(y, p_train, normalize=False)

In [612]:
print(f"Standardization: Yes (StandardScaler used)")
print(f"Intercept: {intercept:.4f}")
print(f"Coefficients: Income={coef[0]:.4f}, Balance={coef[1]:.4f}")
print(f"Training Log-Likelihood: {train_loglik:.4f}")

Standardization: Yes (StandardScaler used)
Intercept: -6.1094
Coefficients: Income=0.2792, Balance=2.7223
Training Log-Likelihood: -789.4877


**2(b) Provide an interpretation of each coefficient on the log-odds scale & interpretation to the odds ratio scale via eβj**

In [613]:
beta_income = coef[0]
beta_balance = coef[1]

odds_income = np.exp(beta_income)
odds_balance = np.exp(beta_balance)

print(f"Income coefficient: {beta_income:.4f}, Odds Ratio = {odds_income:.3f}")
print(f"Balance coefficient: {beta_balance:.4f}, Odds Ratio = {odds_balance:.3f}")

Income coefficient: 0.2792, Odds Ratio = 1.322
Balance coefficient: 2.7223, Odds Ratio = 15.216


**2(c) Code the likelihood and optimize (no closed form)**

In [614]:
from scipy.optimize import minimize

In [615]:
df = pd.read_csv("/Users/lavi/MLDL_25_2_ASSIGNMENT/ASSIGNMENT_1/Data/Default.csv")
df['default_bin'] = df['default'].map({'No':0, 'Yes':1}).astype(int)
X = df[['income','balance']].copy()
y = df['default_bin'].values.astype(float)

In [616]:
RANDOM_SEED = 1
np.random.seed(RANDOM_SEED)

In [617]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000, random_state=RANDOM_SEED))
])
pipe.fit(X, y)
clf = pipe.named_steps['clf']

In [618]:
beta_std_sklearn = np.r_[clf.intercept_[0], clf.coef_[0]]
p_train = pipe.predict_proba(X)[:,1]
train_loglik_sklearn = -log_loss(y, p_train, normalize=False)

In [619]:
scaler = pipe.named_steps['scaler']
X_std = scaler.transform(X)
X_mat = np.column_stack([np.ones(len(X_std)), X_std]).astype(float)


In [620]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-np.clip(z, -35, 35)))

In [621]:
def neg_loglik(beta):
    z = X_mat @ beta
    p = sigmoid(z)
    eps = 1e-12
    return -np.sum(y*np.log(p + eps) + (1 - y)*np.log(1 - p + eps))

In [622]:
#강의자료 Lecture 4, p.13 인용
def grad_neg_loglik(beta):
    z = X_mat @ beta
    p = sigmoid(z)
    return X_mat.T @ (p - y)

In [623]:
beta0 = np.zeros(X_mat.shape[1])
res = minimize(neg_loglik, beta0, jac=grad_neg_loglik, method='BFGS', options={'gtol':1e-8, 'maxiter':1000})
beta_std_mle = res.x
max_loglik_mle = -res.fun

In [624]:
comp = pd.DataFrame({
    'param': ['Intercept','income(std)','balance(std)'],
    'sklearn(2a)': beta_std_sklearn,
    'MLE(c)': beta_std_mle,
    'abs_diff': np.abs(beta_std_sklearn - beta_std_mle)
})
print("=== (a) vs (c) coefficients on standardized scale ===")
print(comp)
print(f"\nTrain log-likelihood (sklearn 2a): {train_loglik_sklearn:.6f}")
print(f"Max log-likelihood (MLE 2c)      : {max_loglik_mle:.6f}")

=== (a) vs (c) coefficients on standardized scale ===
          param  sklearn(2a)    MLE(c)  abs_diff
0     Intercept    -6.109377 -6.125566  0.016190
1   income(std)     0.279158  0.277508  0.001651
2  balance(std)     2.722327  2.731452  0.009125

Train log-likelihood (sklearn 2a): -789.487744
Max log-likelihood (MLE 2c)      : -789.483135


**2(d) Perform a validation set approach(7:3 random val & 5-fold cross val)**

In [625]:
from sklearn.metrics import accuracy_score

df = pd.read_csv("/Users/lavi/MLDL_25_2_ASSIGNMENT/ASSIGNMENT_1/Data/Default.csv")
df['default_bin'] = df['default'].map({'No':0, 'Yes':1}).astype(int)
X = df[['income','balance']]
y = df['default_bin']

RANDOM_SEED = 1
np.random.seed(RANDOM_SEED)

*7:3 random val*

In [626]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=RANDOM_SEED, stratify=y)

In [627]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000, random_state=RANDOM_SEED))
])
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('clf',
                 LogisticRegression(max_iter=1000, penalty=None,
                                    random_state=1))])

In [628]:
proba_val = pipe.predict_proba(X_val)[:, 1]
yhat_val = (proba_val >= 0.5).astype(int)
val_accuracy = accuracy_score(y_val, yhat_val)
val_misclass_rate = 1 - val_accuracy

print("=== Validation Set (70/30 split) ===")
print(f"Accuracy: {val_accuracy:.4f}")
print(f"Misclassification Rate: {val_misclass_rate:.4f}")


=== Validation Set (70/30 split) ===
Accuracy: 0.9760
Misclassification Rate: 0.0240


*5-fold Cross Validation*

In [629]:
cv_scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')
cv_misclass_rate = 1 - cv_scores.mean()

print("\n=== 5-Fold Cross Validation ===")
print(f"Mean Accuracy: {cv_scores.mean():.4f}")
print(f"Mean Misclassification Rate: {cv_misclass_rate:.4f}")


=== 5-Fold Cross Validation ===
Mean Accuracy: 0.9735
Mean Misclassification Rate: 0.0265


**2(e) Include a dummy variable for student Using 5-fold cross-validation**

In [630]:
from sklearn.model_selection import cross_val_score

In [631]:
df['student_bin'] = df['student'].map({'No': 0, 'Yes': 1}).astype(int)

X2 = df[['income', 'balance', 'student_bin']]
y = df['default_bin']

In [632]:
RANDOM_SEED = 1
np.random.seed(RANDOM_SEED)

In [633]:
pipe2 = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000, random_state=RANDOM_SEED))
])

In [634]:
cv_scores_2e = cross_val_score(pipe2, X2, y, cv=5, scoring='accuracy')
cv_misclass_rate_2e = 1 - cv_scores_2e.mean()

print("=== (2e) Logistic Regression with student dummy ===")
print(f"Mean CV Accuracy: {cv_scores_2e.mean():.4f}")
print(f"Mean CV Misclassification Rate: {cv_misclass_rate_2e:.4f}")

=== (2e) Logistic Regression with student dummy ===
Mean CV Accuracy: 0.9732
Mean CV Misclassification Rate: 0.0268
